In [1]:
import pandas as pd

price_cols=['file_guid', 'file_name', 'file_num', 'sheet_name',
      'region','reg_period_start','reg_period_end','tr_date','tr_num','tr_corr_date','tr_corr_num',
      'org','inn','kpp','rst_org_id','mr','mo','oktmo','vdet',
      'nomer','tar_type','unit','tech_osob','ter','tko_type','tko_class',
      'tar_value','volume','date_begin','date_end'
     ]

price_data = pd.read_excel('tko_price_data.xlsx', usecols = price_cols )

# balance_cols=['file_guid', 'file_name', 'file_num', 'sheet_name',
#       'region','report_type','period','org','inn','kpp','rst_org_id','mr','mo','oktmo','vdet',
#       'unit','L2','L3','L4','L5','L7','L8','L9',
#       'L3_1','L3_1_1','L3_1_2','L3_2','L3_2_1','L3_2_2','L3_3','L3_3_1','L3_3_2','L3_4','L3_4_1','L3_4_2',
#       'L3_5','L3_6','L3_7','L3_8',
#       'buh_link'
#      ]

# balance_data = pd.read_excel('tko_balance_data.xlsx', usecols = balance_cols )

price_data['date_begin'] = pd.to_datetime(price_data['date_begin'])
price_data['date_end'] = pd.to_datetime(price_data['date_end'])

regions = pd.read_excel('regions.xlsx', usecols = ['region_id','region_name','fo_name','ord_num','code','fo_ord_num','short_name'] )

In [2]:
data = price_data[(
    (price_data.date_begin <= '01.07.2020')&
    (price_data.date_end >= '31.12.2020')&
    (price_data.tar_value > 0)&
    #(price_data.unit == unit)&
    ( (price_data.tar_type == 'Единый тариф регионального оператора по обращению с твердыми коммунальными отходами')|(price_data.tar_type == 'Тариф на захоронение твердых коммунальных отходов') )
)][['region','org','rst_org_id','unit','tar_type','tech_osob','tko_type','tko_class','tar_value']]
data = data.drop_duplicates(subset = ['region','org','rst_org_id','unit','tar_type','tech_osob','tko_type','tko_class','tar_value'])

data.tko_type.replace(['несортированный','крупногабаритные','нет'], ['несортированные','сортированные','без дифференциации'], inplace=True)
data['region_label'] = data.region.replace(regions.region_name.to_list(), regions.short_name.to_list())
data['region_ord'] = data.region.replace(regions.region_name.to_list(), regions.ord_num.to_list())
data['fo'] = data.region.replace(regions.region_name.to_list(), regions.fo_name.to_list())
data['fo_ord_num'] = data.region.replace(regions.region_name.to_list(), regions.fo_ord_num.to_list())
data['back'] = data.fo_ord_num.replace([1,2,3,4,5,6,7,8,9],[0,1,0,1,0,1,0,1,0])
#data = data.sort_values(by=['fo_ord_num','region_ord','org'])

regions['back'] = regions.fo_ord_num.replace([1,2,3,4,5,6,7,8,9],[0,1,0,1,0,1,0,1,0])
regions = regions.sort_values(by=['fo_ord_num','ord_num'])
regions = regions[regions.fo_ord_num != 9]

In [3]:
print(len(data))
data.head()

1203


,region,org,rst_org_id,unit,tar_type,tech_osob,tko_type,tko_class,tar_value,region_label,region_ord,fo,fo_ord_num,back
84,Республика Мордовия,"МУП ЖКХ ""Ардатовское""",30371263,руб./куб. м,Тариф на захоронение твердых коммунальных отходов,нет,несортированные,IV,19.11,Мордовия,45,Приволжский федеральный округ,5,0
94,Сахалинская область,"МБУ ""Управление городским хозяйством""",26770666,руб./куб. м,Тариф на захоронение твердых коммунальных отходов,Уплотнение и засыпка изоляционным слоем земли,несортированные,IV,21.90,Сахалинская обл.,84,Дальневосточный федеральный округ,8,1
100,Курганская область,ИП Некрутов В.А.,31192399,руб./куб. м,Тариф на захоронение твердых коммунальных отходов,без дифференциации,без дифференциации,без дифференциации,23.28,Курганская обл.,57,Уральский федеральный округ,6,1
102,Сахалинская область,МУП «ЖКХ УГО»,30354012,руб./куб. м,Тариф на захоронение твердых коммунальных отходов,без дифференциации,без дифференциации,без дифференциации,23.68,Сахалинская обл.,84,Дальневосточный федеральный округ,8,1
115,Сахалинская область,"МБУ ""Управление городским хозяйством""",26770666,руб./куб. м,Тариф на захоронение твердых коммунальных отходов,Уплотнение и засыпка изоляционным слоем земли,несортированные,IV,25.84,Сахалинская обл.,84,Дальневосточный федеральный округ,8,1


In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

max_r = 30000
min_r = 10

fig = make_subplots(
    rows = 2, cols = 3,
    column_widths=[0.04, 0.48, 0.48],
    row_heights=[0.68, 0.32],
    vertical_spacing = 0.05, horizontal_spacing = 0.1,
    specs=[[None, {'type': 'polar'}, {'type': 'polar'}],
           [None, {"type": "table"}, {"type": "table"} ]],
    subplot_titles=('Тарифы за объём (руб./куб.м)',
                    'Тарифы за массу (руб./тонна)',
                    'Наибольшие и наименьшие тарифы за объём ТКО',
                    'Наибольшие и наименьшие тарифы за массу ТКО')
)

marker_size = 11

tko_types = [
    {'name': 'без дифференциации', 'color': 'gray', 'marker_color': 'rgba(100,100,100,0.5)', 'line_color': 'rgb(100,100,100)'},
    {'name': 'сортированные', 'color': 'green', 'marker_color': 'rgba(15,188,15,0.5)', 'line_color': 'rgb(15,188,15)'},
    {'name': 'несортированные', 'color': 'red', 'marker_color': 'rgba(250,20,20,0.5)', 'line_color': 'rgb(250,20,20)'},
]

tko_classes = [
    {'name': 'без дифференциации', 'size' : marker_size},
    {'name': 'I', 'size' : marker_size},
    {'name': 'II', 'size' : marker_size+1},
    {'name': 'III', 'size' : marker_size+2},
    {'name': 'IV', 'size' : marker_size+4},
    {'name': 'V', 'size' : marker_size+6},
]

tar_types = [
    {'name': 'Тариф на захоронение твердых коммунальных отходов',
     'symbol': 'circle', 'label': 'Тариф на захоронение ТКО'},
    {'name': 'Единый тариф регионального оператора по обращению с твердыми коммунальными отходами',
     'symbol': 'diamond', 'label': 'Тариф регионального оператора'},
]

units = [
    {'name': 'руб./куб. м', 'num': 2},
    {'name': 'руб./тонна', 'num': 3}
]

for unit in units:
    for tar_type in tar_types:
        for tko_type in tko_types:
            for tko_class in tko_classes:
                df = data[(data.tko_type == tko_type['name'])&\
                          (data.tko_class == tko_class['name'])&\
                          (data.tar_type == tar_type['name'])&\
                          (data.unit == unit['name'])&\
                          (data.tar_value <= max_r)&(data.tar_value >= min_r)
                         ]
                fig.add_trace(go.Scatterpolar(
                    name = tar_type['label']+', тип - '+tko_type['name']+', класс опасности - '+tko_class['name'],
                    r = df['tar_value'], theta = df['region_label'],
                    mode = 'markers',
                    marker = dict(color = tko_type['marker_color'],
                                  line = dict(width = 0.5, color = tko_type['line_color']),
                                  size = tko_class['size'],
                                  symbol = tar_type['symbol']),
                    opacity = 0.6,
                    showlegend = False,
                ), row = 1, col = unit['num'])

    fig.add_trace(go.Barpolar(
        name = 'back1',
        r = regions['back']*max_r, theta = regions['short_name'], width = 1,
        marker_color = 'rgba(150,150,150,150)', marker_line_color = 'rgba(0,0,0,0)',
        opacity = 0.2,
        showlegend = False
    ), row = 1, col = unit['num'])

    fig.add_trace(go.Barpolar(
        name = 'back2',
        r = (1-regions['back'])*max_r, theta = regions['short_name'], width = 1,
        marker_color = 'rgba(220,220,220,220)', marker_line_color = 'rgba(0,0,0,0)',
        opacity = 0.2,
        showlegend = False
    ), row = 1, col = unit['num'])

# legend
for tar_type in tar_types:
    fig.add_trace(go.Scatterpolar(
        name = tar_type['label'], r = [1]*len(regions), theta = regions['short_name'],
        mode = 'markers',
        marker = dict( color = 'white', line = dict(width = 0.5, color = 'grey'), size = 12, symbol = tar_type['symbol']),
        showlegend = True,
    ), row = 1, col = 2)
    
for tko_class in tko_classes:
    fig.add_trace(go.Scatterpolar(
        name = 'Класс опасности - '+tko_class['name'], r = [1]*len(regions), theta = regions['short_name'],
        mode = 'markers',
        marker = dict( color = 'white', line = dict(width = 0.5, color = 'grey'), size = tko_class['size'], symbol = 'circle'),
        showlegend = True,
    ), row = 1, col = 2)

for tko_type in tko_types:
    fig.add_trace(go.Scatterpolar(
        name = 'Тип - '+tko_type['name'], r = [1]*len(regions), theta = regions['short_name'],
        mode = 'markers',
        marker = dict( color = tko_type['marker_color'], line = dict(width = 0.5, color = tko_type['line_color']), size = 12, symbol = 'circle'),
        showlegend = True,
    ), row = 1, col = 2)


# tables
for unit in units:
    d_min = data[data.unit == unit['name']][['region','org','tar_value']].\
            groupby(['region','org']).min().\
            sort_values(by=['tar_value'], ascending = True).\
            reset_index().head(5)
    d_max = data[data.unit == unit['name']][['region','org','tar_value']].\
            groupby(['region','org']).max().\
            sort_values(by=['tar_value'], ascending = False).\
            reset_index().head(5).\
            sort_values(by=['tar_value'], ascending = True)
    
    d = pd.concat([d_min, pd.DataFrame(data = {'region': ['. . .'], 'org': ['. . .'], 'tar_value': ['. . .']}), d_max])

    fig.add_trace(go.Table(
        columnwidth = [250, 50],
        header=dict(
            values = ['<b>Регион, организация</b><br>','<b>Тариф,<br>'+unit['name']+'</b>'],
            fill_color = 'rgb(220,220,220)', align = ['left', 'left'], height = 25
        ),
        cells=dict(
            values=[(d.region+', '+d.org).replace('. . ., . . .', '. . .'), d.tar_value],
            fill_color = 'rgb(250,250,250)', align = ['left', 'right'], font_size = 9, height = 20,
        )
    ), row = 2, col = unit['num'])


# layout
fig_lo = dict(
    bgcolor = 'rgba(0,0,0,0)',
    radialaxis = dict(
        type = 'log', range = [math.log(min_r,10),math.log(max_r,10)],
        showline = False,
        gridcolor = 'rgba(240,240,240,240)', gridwidth = 1
    ),
    angularaxis = dict(
        ticks = 'outside', tickcolor = 'rgba(240,240,240,240)', tickfont = dict(size = 6),
        showline = True, linecolor = 'rgba(240,240,240,240)', linewidth = 0.5,
        gridcolor = 'rgba(240,240,240,240)', gridwidth = 0.5,
        rotation = 0, direction = "clockwise"
    ),
    angularaxis_categoryarray = regions.sort_values(by=['fo_ord_num','ord_num'])['short_name'].to_list()
    )

fig.update_layout(
    font = dict(size = 10, family = 'Helvetica', color = 'rgb(100,100,100)'),
    showlegend = True,
    polar = fig_lo,
    polar2 = fig_lo,
    title = dict(text = 'Тарифы в области обращения с твердыми коммунальными отходами на II полугодие 2020 года',
                 font = dict(size = 20), x = 0.01),
    paper_bgcolor = 'rgba(0,0,0,0)',
#     annotations = [
#         dict(x = 0.05, y = 0.30, xref = 'paper', yref = 'paper',
#              align = 'left', valign = 'top',
#              text = a_text, showarrow = False,
#              font = dict(size = 10)),
#         dict(x = 0.2, y = -0.05, xref = 'paper', yref = 'paper',
#              align = 'left', valign = 'top',
#              text = 'Источники: раскрытие информации ресурсоснабжающими организациями, постановления об утверждении тарифов региональных органов регулирования',
#              showarrow = False,
#              font = dict(size = 10))
#     ],
)

    
# annotation
a_text =\
    'Информация по тарифам в сфере обращения с ТКО<br>на II полугодие 2020 года <b>раскрыта</b><br>' +\
    'по ' + str(data.agg({'region': 'nunique', 'org': 'nunique'})['org']) + ' организации<br>'+\
    'из ' + str(data.agg({'region': 'nunique', 'org': 'nunique'})['region']) + ' регионов.<br>'+\
    '<br><br>' +\
    'Субъекты, по которым информация<br><b>не раскрыта</b> ни по одной орагнизации:<br>' +\
    '- '+',<br>- '.join(set(regions['region_name'].unique()) - set(price_data[((price_data.date_begin <= '01.07.2020')&(price_data.date_end >= '31.12.2020')&(price_data.tar_value > 0))]['region'].unique()))+'<br>'+\
    '<br><br>' +\
    'Тарифы с дифференциацией по<br><b>типу ТКО</b> установлены<br>' +\
    'по ' + str(data[(data.tko_type != 'без дифференциации')].agg({'region': 'nunique', 'org': 'nunique'})['org']) + ' организациям<br>'+\
    'из ' + str(data[(data.tko_type != 'без дифференциации')].agg({'region': 'nunique', 'org': 'nunique'})['region']) + ' регионов.<br>'+\
    '<br><br>' +\
    'Тарифы с дифференциацией по<br><b>классу опасности ТКО</b> установлены<br>' +\
    'по ' + str(data[(data.tko_class != 'без дифференциации')].agg({'region': 'nunique', 'org': 'nunique'})['org']) + ' организациям<br>'+\
    'из ' + str(data[(data.tko_class != 'без дифференциации')].agg({'region': 'nunique', 'org': 'nunique'})['region']) + ' регионов.<br>'+\
    '<br><br>' +\
    'Субъекты, в которых <b>отсутствует<br>дифференциация</b> по типу<br>или классу опасности ТКО:<br>' +\
    '- '+',<br>- '.join(set(data[(data.tar_value > 0)]['region'].unique()) - set(data[(data.tko_class != 'без дифференциации')|(data.tko_type != 'без дифференциации')]['region'].unique()))+'<br>'+\
    '<br>'

annotations = [a.to_plotly_json() for a in fig['layout']['annotations']]
annotations.append({'font': {'size': 10},
  'showarrow': False,'text': a_text,
  'x': -0.03,'y': 0.3, 'align': 'left', 'valign': 'top', 'xref': 'paper', 'yref': 'paper'})
annotations.append({'font': {'size': 10},
  'showarrow': False,'text': 'Источники: раскрытие информации ресурсоснабжающими организациями, постановления об утверждении тарифов региональных органов регулирования',
  'x': -0.03,'y': -0.05, 'align': 'left', 'valign': 'top', 'xref': 'paper', 'yref': 'paper'})
fig['layout']['annotations'] = annotations


fig.show()

In [19]:
import plotly.io as pio

file_format = 'pdf'
pio.write_image(fig, 'tko_tar_2020.'+file_format, format = file_format, width=1920, height=1200)


In [23]:
?pio.write_image

### Статистика

In [67]:
data.agg({'region': 'nunique', 'org': 'nunique'})

region     80
org       601
dtype: int64

In [61]:
data[(data.tko_type != 'без дифференциации')].agg({'region': 'nunique', 'org': 'nunique'})

region     56
org       199
dtype: int64

In [62]:
data[(data.tko_class != 'без дифференциации')].agg({'region': 'nunique', 'org': 'nunique'})

region     55
org       209
dtype: int64

In [94]:
data[(data.tko_class != 'без дифференциации')&(data.tko_type != 'без дифференциации')].agg({'region': 'nunique', 'org': 'nunique'})

region     55
org       168
dtype: int64

### ТОП организаций по величине тарифа

In [12]:
data[['region','org','tar_value']].\
    groupby(['region','org']).min().\
    sort_values(by=['tar_value'], ascending = True).\
    head(10)

,,tar_value
region,org,
Республика Мордовия,"МУП ЖКХ ""Ардатовское""",19.11
Сахалинская область,"МБУ ""Управление городским хозяйством""",21.90
Курганская область,ИП Некрутов В.А.,23.28
Сахалинская область,МУП «ЖКХ УГО»,23.68
Республика Мордовия,"МУП ""Энергосервис""",31.03
Самарская область,"Государственное унитарное предприятие Самарской области ""Экология""",33.77
Курганская область,"МУП ""Шуховский полигон ТБО""",36.00
Ростовская область,"ООО ""Комбинат коммунальных предприятий""",36.76
Нижегородская область,"МУП ""КОМУНЭНЕРГО""",40.55


In [13]:
data[['region','org','tar_value']].\
    groupby(['region','org']).max().\
    sort_values(by=['tar_value'], ascending = False).\
    head(10)

tar_value
region                            org                                                          
Ненецкий автономный округ         МП ЗР "Севержилкомсервис"                            26587.47
Чукотский автономный округ        МУП ЖКХ "Иультинское"                                14925.15
Ямало-Ненецкий автономный округ   филиал - Управление по эксплуатации вахтовых по...   10925.00
Чукотский автономный округ        ООО "РТК"                                            10172.66
                                  МП "ЧРКХ"                                             6351.39
                                  ООО "Андезит"                                         5613.93
                                  ООО "Сервис Групп"                                    5284.19
                                  ООО "Полигон"                                         4573.39
Ханты-Мансийский автономный округ Муниципальное предприятие "ЖЭК-3" Ханты-Мансийс...    2194.17
Чукотский автономный округ        ООО "АТК"                                             1492.05

## Проверка данных

In [12]:
# Тарифы за куб.м
d1 = price_data[((price_data.date_begin <= '01.07.2020')&(price_data.date_end >= '31.12.2020')&(price_data.unit == 'руб./куб. м'))]

# Тарифы за тонну
d2 = price_data[((price_data.date_begin <= '01.07.2020')&(price_data.date_end >= '31.12.2020')&(price_data.unit == 'руб./тонна'))]

r1 = set(d1['region'])
r2 = set(d2['region'])
r = set(regions['region_name'])
len(r1), len(r2), len(r)

(72, 57, 85)

In [13]:
# регионы, по которым нет данных
r - r1 - r2

{'Карачаево-Черкесская республика',
 'Республика Ингушетия',
 'Республика Тыва',
 'г.Севастополь'}

In [42]:
# ТОП наибольших тарифов за куб.м
d1.sort_values(by=['tar_value'], ascending=False).head(10)

,file_guid,file_name,file_num,sheet_name,region,reg_period_start,reg_period_end,tr_date,tr_num,tr_corr_date,...,tar_type,unit,tech_osob,ter,tko_type,tko_class,tar_value,volume,date_begin,date_end
9,9a705a6b-a1bb-5aae-e053-920aa8c0da64,9a705a6b-a1bb-5aae-e053-920aa8c0da64 001_FAS.J...,390,Форма 5.3.2,Ненецкий автономный округ,01.01.2020,31.12.2020,19.12.2019,89,NaN,...,NaN,руб./куб. м,без дифференциации,Прочие потребители (море) Канинский с/с Карски...,без дифференциации,без дифференциации,26587.47,NaN,2020-01-07,2020-12-31
12,9a705a6b-a1bb-5aae-e053-920aa8c0da64,9a705a6b-a1bb-5aae-e053-920aa8c0da64 001_FAS.J...,390,Форма 5.3.2,Ненецкий автономный округ,01.01.2020,31.12.2020,19.12.2019,89,NaN,...,NaN,руб./куб. м,без дифференциации,Прочие потребители (море) Канинский с/с Карски...,без дифференциации,без дифференциации,26587.47,NaN,2020-01-07,2020-12-31
13,9a705a6b-a1bb-5aae-e053-920aa8c0da64,9a705a6b-a1bb-5aae-e053-920aa8c0da64 001_FAS.J...,390,Форма 5.3.2,Ненецкий автономный округ,01.01.2020,31.12.2020,19.12.2019,89,NaN,...,NaN,руб./куб. м,без дифференциации,Прочие потребители (море) Канинский с/с Карски...,без дифференциации,без дифференциации,26587.47,NaN,2020-01-07,2020-12-31
17,9a705a6b-a1bb-5aae-e053-920aa8c0da64,9a705a6b-a1bb-5aae-e053-920aa8c0da64 001_FAS.J...,390,Форма 5.3.2,Ненецкий автономный округ,01.01.2020,31.12.2020,19.12.2019,89,NaN,...,NaN,руб./куб. м,без дифференциации,Прочие потребители (море) Канинский с/с Карски...,без дифференциации,без дифференциации,26587.47,NaN,2020-01-07,2020-12-31
27,9a705a6b-a1bb-5aae-e053-920aa8c0da64,9a705a6b-a1bb-5aae-e053-920aa8c0da64 001_FAS.J...,390,Форма 5.3.2,Ненецкий автономный округ,01.01.2020,31.12.2020,19.12.2019,89,NaN,...,NaN,руб./куб. м,без дифференциации,Прочие потребители (море) Канинский с/с Карски...,без дифференциации,без дифференциации,26587.47,NaN,2020-01-07,2020-12-31
33,9a705a6b-a1bb-5aae-e053-920aa8c0da64,9a705a6b-a1bb-5aae-e053-920aa8c0da64 001_FAS.J...,390,Форма 5.3.2,Ненецкий автономный округ,01.01.2020,31.12.2020,19.12.2019,89,NaN,...,NaN,руб./куб. м,без дифференциации,Прочие потребители (море) Канинский с/с Карски...,без дифференциации,без дифференциации,26587.47,NaN,2020-01-07,2020-12-31
40,9a705a6b-a1bb-5aae-e053-920aa8c0da64,9a705a6b-a1bb-5aae-e053-920aa8c0da64 001_FAS.J...,390,Форма 5.3.2,Ненецкий автономный округ,01.01.2020,31.12.2020,19.12.2019,89,NaN,...,NaN,руб./куб. м,без дифференциации,Прочие потребители (море) Канинский с/с Карски...,без дифференциации,без дифференциации,26587.47,NaN,2020-01-07,2020-12-31
42,9a705a6b-a1bb-5aae-e053-920aa8c0da64,9a705a6b-a1bb-5aae-e053-920aa8c0da64 001_FAS.J...,390,Форма 5.3.2,Ненецкий автономный округ,01.01.2020,31.12.2020,19.12.2019,89,NaN,...,NaN,руб./куб. м,без дифференциации,Прочие потребители (море) Канинский с/с Карски...,без дифференциации,без дифференциации,26587.47,NaN,2020-01-07,2020-12-31
10,9a705a6b-a1bb-5aae-e053-920aa8c0da64,9a705a6b-a1bb-5aae-e053-920aa8c0da64 001_FAS.J...,390,Форма 5.3.2,Ненецкий автономный округ,01.01.2020,31.12.2020,19.12.2019,89,NaN,...,NaN,руб./куб. м,без дифференциации,Прочие потребители (море) Канинский с/с Карски...,без дифференциации,без дифференциации,26587.47,NaN,2020-01-07,2020-12-31
74,72fde635-ccd2-48f8-b5ef-0bbaa65f31fa,72fde635-ccd2-48f8-b5ef-0bbaa65f31fa 002_FAS.J...,779,Форма 5.3.2,Чукотский автономный округ,01.01.2019,31.12.2021,30.11.2018,20-к/7,NaN,...,NaN,руб./куб. м,без дифференциации,Участок Ванкарем,без дифференциации,без дифференциации,14925.15,NaN,2020-01-07,2020-12-31


In [43]:
# ТОП наименьших тарифов за куб.м
d1.sort_values(by=['tar_value'], ascending=True).head(10)

,file_guid,file_name,file_num,sheet_name,region,reg_period_start,reg_period_end,tr_date,tr_num,tr_corr_date,...,tar_type,unit,tech_osob,ter,tko_type,tko_class,tar_value,volume,date_begin,date_end
13600,4dfc05c4-cc3b-455d-9634-13d3a9792299,4dfc05c4-cc3b-455d-9634-13d3a9792299 001_FAS.J...,718,Форма 5.3.2,Архангельская область,01.01.2018,31.12.2020,01.12.2017,68-в/6,NaN,...,NaN,руб./куб. м,без дифференциации,тариф на захоронение ТКО,без дифференциации,без дифференциации,0.00,NaN,2020-01-01,2020-12-31
13596,9aea5467-59fe-cada-e053-8d78a8c05a4a,9aea5467-59fe-cada-e053-8d78a8c05a4a 001_FAS.J...,653,Форма 5.3.2,Нижегородская область,01.01.2020,31.12.2020,20.12.2019,65/51,NaN,...,NaN,руб./куб. м,без дифференциации,-,сортированные,без дифференциации,15.19,NaN,2020-01-01,2020-12-31
13595,0147cd25-fb1a-49ed-b4fa-f107f7d7fc15,0147cd25-fb1a-49ed-b4fa-f107f7d7fc15 001_FAS.J...,1008,Форма 5.3.2,Республика Мордовия,01.04.2019,31.12.2021,14.03.2019,13,NaN,...,NaN,руб./куб. м,нет,"Ардатовский муниципальный район, Атяшевский ра...",несортированные,IV,19.11,NaN,2019-01-07,2021-12-31
13585,f084bcdf-be45-4c68-b106-0af1d3fc8d9b,f084bcdf-be45-4c68-b106-0af1d3fc8d9b 001_FAS.J...,598,Форма 5.3.2,Сахалинская область,01.01.2018,31.12.2020,28.11.2017,Приказ №44-ОК,10.12.2018,...,NaN,руб./куб. м,Уплотнение и засыпка изоляционным слоем земли,"Городской округ ""Долинский""",несортированные,IV,21.90,NaN,2020-01-07,2020-12-31
13578,4b1acb44-5414-4e22-81c4-5f459677893a,4b1acb44-5414-4e22-81c4-5f459677893a 001_FAS.J...,1316,Форма 5.3.2,Курганская область,01.01.2020,31.12.2022,20.12.2019,45-101,NaN,...,NaN,руб./куб. м,без дифференциации,Захорнение ТКО,без дифференциации,без дифференциации,23.28,NaN,2020-01-01,2020-12-31
13577,1f49490f-2d21-4eb6-bf75-a6611a20da11,1f49490f-2d21-4eb6-bf75-a6611a20da11 001_FAS.J...,1001,Форма 5.3.2,Сахалинская область,01.01.2018,31.12.2020,19.12.2018,92-ОКК,NaN,...,NaN,руб./куб. м,без дифференциации,г. Углегорск,без дифференциации,без дифференциации,23.68,NaN,2020-01-07,2020-12-31
13557,f084bcdf-be45-4c68-b106-0af1d3fc8d9b,f084bcdf-be45-4c68-b106-0af1d3fc8d9b 001_FAS.J...,598,Форма 5.3.2,Сахалинская область,01.01.2018,31.12.2020,28.11.2017,Приказ №44-ОК,10.12.2018,...,NaN,руб./куб. м,Уплотнение и засыпка изоляционным слоем земли,"Городской округ ""Долинский""",несортированные,IV,25.84,NaN,2019-01-07,2020-12-31
13550,1f49490f-2d21-4eb6-bf75-a6611a20da11,1f49490f-2d21-4eb6-bf75-a6611a20da11 001_FAS.J...,1001,Форма 5.3.2,Сахалинская область,01.01.2018,31.12.2020,19.12.2018,92-ОКК,NaN,...,NaN,руб./куб. м,без дифференциации,г. Углегорск,без дифференциации,без дифференциации,27.61,NaN,2020-01-07,2020-12-31
13543,c9c47c2c-fca7-4ba1-b350-fce8ac5978fa,c9c47c2c-fca7-4ba1-b350-fce8ac5978fa 001_FAS.J...,1175,Форма 5.3.2,Курская область,01.01.2018,31.12.2020,23.10.2017,32,21.11.2019,...,NaN,руб./куб. м,без дифференциации,"28,23руб/куб",без дифференциации,без дифференциации,28.79,NaN,2020-01-07,2020-12-31
13532,9eba358a-e796-4aa1-9f13-eb1a41a8cfbe,9eba358a-e796-4aa1-9f13-eb1a41a8cfbe 001_FAS.J...,1176,Форма 5.3.2,Курская область,01.01.2019,31.12.2029,09.11.2018,134,NaN,...,NaN,руб./куб. м,без дифференциации,"27,93 руб/м2",без дифференциации,без дифференциации,29.48,NaN,2020-01-07,2020-12-31


In [48]:
# ТОП наибольших тарифов за тонну
d2.sort_values(by=['tar_value'], ascending=False)\
    [['file_guid','region','tr_date','tr_num','tr_corr_date','tr_corr_num','org',
      'tech_osob','ter','tko_type','tko_class','tar_value','date_begin','date_end']].head(10)

,file_guid,region,tr_date,tr_num,tr_corr_date,tr_corr_num,org,tech_osob,ter,tko_type,tko_class,tar_value,date_begin,date_end
11,bad068bf-9101-4cca-96b8-e5249b4650d4,Новосибирская область,12.12.2019,№ 622-ЖКХ,NaN,NaN,"Общество с ограниченной ответственностью ""СибР...","Обезвреживание отходов с помощью установки ""Эк...",Разделение отходов на состовляющие и их дальне...,сортированные,I,75841.55,2020-01-07,2020-12-31
100,9c56bd7d-1ac0-f43b-e053-8d78a8c00e89,Ямало-Ненецкий автономный округ,20.12.2019,408-т,NaN,NaN,"АО ""Ямалэкосервис""",без дифференциации,с. Яр-Сале,без дифференциации,без дифференциации,25913.80,2020-01-07,2020-12-31
106,f09e945c-4a2d-45d2-a4cf-27dedbf63f5f,Ханты-Мансийский автономный округ,13.12.2018,103-нп,NaN,NaN,"Муниципальное предприятие ""ЖЭК-3"" Ханты-Мансий...","Тариф, руб/тонну",сп. Кедровый с. Елизарово,без дифференциации,без дифференциации,25004.54,2020-01-07,2020-12-31
112,69e4931c-8146-4f9b-ab60-6779bed57e97,Свердловская область,30.11.2018,198-ПК,NaN,NaN,"Общество с ограниченной ответственностью ""Экот...",без дифференциации,x,без дифференциации,IV,23883.31,2020-01-07,2020-12-31
118,69e4931c-8146-4f9b-ab60-6779bed57e97,Свердловская область,30.11.2018,198-ПК,NaN,NaN,"Общество с ограниченной ответственностью ""Экот...",без дифференциации,x,без дифференциации,V,23607.40,2020-01-07,2020-12-31
13767,9fdffa77-5557-7758-e053-8d78a8c09970,Ямало-Ненецкий автономный округ,31.01.2020,5-Т,NaN,NaN,"ООО НПП ""СГТ""",без дифференциации,нет,без дифференциации,без дифференциации,15985.50,2020-01-02,2020-12-31
132,2c23a28c-8cc2-4397-8b9a-8911d3d9061a,Ханты-Мансийский автономный округ,18.02.2020,4-нп,NaN,NaN,"Акционерное общество ""Югра-Экология""","Источник образования ТКО и отходообразователи,...","Кондинский район, Нефтеюганский район, Октябрь...",без дифференциации,без дифференциации,14616.80,2020-01-07,2020-12-31
272,bf90f784-a163-401a-82c7-b5c747c96de6,Ханты-Мансийский автономный округ,14.11.2019,№ 86-нп,NaN,NaN,"Муниципальное предприятие ""ЖЭК-3"" Ханты-Мансий...",Тариф руб./тонну,сп.Кышик (с.Кышик),без дифференциации,без дифференциации,8755.79,2020-01-07,2020-12-31
296,f74e3780-a843-4938-bacc-bed89ca8ce5d,Ханты-Мансийский автономный округ,19.12.2019,166-нп,NaN,NaN,"Акционерное общество ""Югра-Экология""",без дифференциации,"Березовский район, Белоярский район, Сургутски...",без дифференциации,без дифференциации,6846.59,2020-01-07,2020-12-31
322,f74e3780-a843-4938-bacc-bed89ca8ce5d,Ханты-Мансийский автономный округ,19.12.2019,166-нп,NaN,NaN,"Акционерное общество ""Югра-Экология""",без дифференциации,"Березовский район, Белоярский район, Сургутски...",без дифференциации,без дифференциации,5705.49,2020-01-07,2020-12-31


In [45]:
# ТОП наименьших тарифов за тонну
d2.sort_values(by=['tar_value'], ascending=True).head(10)

,file_guid,file_name,file_num,sheet_name,region,reg_period_start,reg_period_end,tr_date,tr_num,tr_corr_date,...,tar_type,unit,tech_osob,ter,tko_type,tko_class,tar_value,volume,date_begin,date_end
1633,b9561bd4-5466-4364-afed-3a10e36012f1,b9561bd4-5466-4364-afed-3a10e36012f1 002_FAS.J...,122,Форма 5.3.2,Иркутская область,01.01.2020,31.12.2020,29.10.2018,110-37-1151-18,NaN,...,NaN,руб./тонна,без дифференциации,Промышленная площадка,без дифференциации,без дифференциации,0.00,NaN,2020-01-01,2020-12-31
1147,9afc8fac-dc66-402e-e053-8d78a8c0bcf8,9afc8fac-dc66-402e-e053-8d78a8c0bcf8 001_FAS.J...,678,Форма 5.3.2,Волгоградская область,01.01.2020,31.12.2020,20.12.2019,44/4,NaN,...,NaN,руб./тонна,-,-,сортированные,IV,49.07,NaN,2020-01-01,2020-12-31
10548,289f06cd-716a-47ee-9e6b-9461afda9157,289f06cd-716a-47ee-9e6b-9461afda9157 001_FAS.J...,578,Форма 5.3.2,Республика Северная Осетия-Алания,01.01.2017,31.12.2021,29.12.2018,58,NaN,...,NaN,руб./тонна,без дифференциации,город Владикавказ,без дифференциации,без дифференциации,62.71,NaN,2020-01-07,2020-12-31
10210,0216485e-214b-4cd4-871d-a09a8c76228a,0216485e-214b-4cd4-871d-a09a8c76228a 001_FAS.J...,487,Форма 5.3.2,Курская область,01.01.2018,31.12.2020,02.11.2018,106,NaN,...,NaN,руб./тонна,без дифференциации,МО г.Железногорск Курской области,крупногабаритные,IV,85.14,NaN,2020-01-07,2020-12-31
2110,9c4e9110-f780-865d-e053-8d78a8c00e2e,9c4e9110-f780-865d-e053-8d78a8c00e2e 001_FAS.J...,325,Форма 5.3.2,Кемеровская область,01.01.2020,31.12.2020,28.11.2019,516,NaN,...,NaN,руб./тонна,без дифференциации,Киселевский городской округ,без дифференциации,без дифференциации,95.27,NaN,2020-01-01,2020-12-31
2104,9d424485-92c8-a3e0-e053-8d78a8c0f7ac,9d424485-92c8-a3e0-e053-8d78a8c0f7ac 001_FAS.J...,323,Форма 5.3.2,Кемеровская область,01.01.2020,31.12.2020,28.11.2019,515,NaN,...,NaN,руб./тонна,без дифференциации,полигон ТБО п.Карагайлинский,без дифференциации,без дифференциации,111.35,NaN,2020-01-01,2020-12-31
9476,a149367e-d834-4a3a-abc4-554a372def20,a149367e-d834-4a3a-abc4-554a372def20 001_FAS.J...,599,Форма 5.3.2,Сахалинская область,01.01.2020,31.12.2020,28.11.2017,№44-ОКК,20.07.2020,...,NaN,руб./тонна,Уплотнение и засыпка изоляционным слоем земли,"Городской округ ""Долинский""",несортированные,IV,111.36,NaN,2020-01-07,2020-12-31
8964,0f63d080-a87d-43ac-8324-6bb9242747ac,0f63d080-a87d-43ac-8324-6bb9242747ac 001_FAS.J...,227,Форма 5.3.2,Свердловская область,01.01.2020,31.12.2020,29.11.2019,147-ПК,NaN,...,NaN,руб./тонна,без дифференциации,Новоуральский городской округ,без дифференциации,V,141.50,NaN,2020-01-07,2020-12-31
8952,98f03c0a-91e9-241e-e053-322b210aaaab,98f03c0a-91e9-241e-e053-322b210aaaab 002_FAS.J...,509,Форма 5.3.2,Алтайский край,01.01.2020,31.12.2020,27.11.2019,391,NaN,...,NaN,руб./тонна,-,-,несортированные,V,143.16,NaN,2020-01-07,2020-12-31
8944,71dcf743-8d2a-49af-82db-c6f002ba1b8e,71dcf743-8d2a-49af-82db-c6f002ba1b8e 001_FAS.J...,871,Форма 5.3.2,Свердловская область,01.01.2020,31.12.2020,29.11.2019,147-ПК,NaN,...,NaN,руб./тонна,уплотнение и засыпка изолдяционным слоем земли,г. Красноуральск,несортированные,V,143.71,NaN,2020-01-07,2020-12-31
